In [1]:
!pip install transformers

In [2]:
pip install accelerate -U

In [3]:
!pip install transformers[torch]

In [4]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
from tokenizers import Tokenizer
from typing import Dict, List, Optional
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from IPython.display import display
from typing import Dict

In [5]:
df = pd.read_csv("/content/drive/MyDrive/bible/pair.csv")

In [6]:
df.describe()

,input_text,target_text
count,31092,31092
unique,30844,30683
top,여호와께서 모세에게 말씀하셨습니다.,여호와께서 모세에게 일러 가라사대
freq,69,65


In [8]:
df.isnull().sum() # 결측치는 없음

input_text     0
target_text    0
dtype: int64

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
model_name = "gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [11]:
style_map={
    'input_text':'문어체',
    'target_text':'성경체'
}

In [12]:
class TextStyleTransferDataset(Dataset):
  def __init__(self,
               df: pd.DataFrame,
               tokenizer: Tokenizer
               ):
    self.df = df
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    row = self.df.iloc[index, :].dropna().sample(2)
    text1 = row[0]
    text2 = row[1]
    target_style = row.index[1]
    target_style_name = style_map[target_style]

    encoder_text = f"{target_style_name} 말투로 변환:{text2}"
    decoder_text = f"{text1}{self.tokenizer.eos_token}"
    model_inputs = self.tokenizer(encoder_text, max_length=64, truncation=True)

    with self.tokenizer.as_target_tokenizer():
      labels = tokenizer(decoder_text, max_length=64, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    del model_inputs['token_type_ids']

    return model_inputs

In [13]:
df.iloc[0]

input_text     태초에 하나님께서 하늘과 땅을 창조하셨습니다.
target_text          태초에 하나님이 천지를 창조하시니라
Name: 0, dtype: object

In [14]:
row_test=df.iloc[0, :].dropna().sample(2)

In [15]:
text1=row_test[0]
text1

'태초에 하나님이 천지를 창조하시니라'

In [16]:
text2=row_test[1]
text2

'태초에 하나님께서 하늘과 땅을 창조하셨습니다.'

In [17]:
target_style = row_test.index[1]

In [18]:
target_style_name = style_map[target_style]

In [19]:
dataset = TextStyleTransferDataset(df, tokenizer)

In [20]:
out = dataset[0]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [21]:
print(out['input_ids'])
print(out['labels'])

[14127, 9085, 12687, 14070, 13282, 10338, 14296, 13716, 257, 13145, 12717, 11786, 14392, 16613, 14578, 14404, 17034, 18940, 26418]
[14438, 12717, 11786, 20806, 17429, 16487, 9120, 21665, 17034, 20032, 15170, 1]


In [22]:
print(tokenizer.decode(out['input_ids']))
print(tokenizer.decode(out['labels']))

성경체 말투로 변환:태초에 하나님이 천지를 창조하시니라
태초에 하나님께서 하늘과 땅을 창조하셨습니다.</s>


In [23]:
out = dataset[1]
print(out['input_ids'])
print(out['labels'])
print(tokenizer.decode(out['input_ids']))
print(tokenizer.decode(out['labels']))

[14111, 11763, 12687, 14070, 13282, 10338, 14296, 13716, 257, 9229, 10281, 9828, 14028, 15495, 12005, 14446, 14400, 17345, 12048, 11229, 14082, 16763, 14202, 18618, 21686, 14392, 14807, 1700, 13185, 14085, 11763, 14418, 15170, 22504, 12034, 17433, 29253, 18896, 9102, 21512, 21420, 14126, 12005, 14198, 20341, 15549, 14502, 14158, 11306, 15170]
[15495, 12034, 14929, 9869, 14058, 14061, 13644, 14363, 16816, 11711, 12034, 15397, 12013, 16782, 15438, 21420, 14094, 12005, 14032, 18466, 19808, 18940, 26418, 1]
문어체 말투로 변환:그런데 그 땅은 지금처럼 짜임새 있는 모습이 아니었고, 생물 하나 없이 텅 비어 있었습니다. 어둠이 깊은 바다를 덮고 있었고, 하나님의 영은 물 위에서 움직이고 계셨습니다.
땅이 혼돈하고 공허하며 흑암이 깊음 위에 있고 하나님의 신은 수면에 운행하시니라</s>


In [24]:
from sklearn.model_selection import train_test_split

# 학습을 위해 train, test set으로 나눈다.
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
print(len(df_train), len(df_test))

27982 3110


In [25]:
train_dataset = TextStyleTransferDataset(
    df_train,
    tokenizer
)
test_dataset = TextStyleTransferDataset(
    df_test,
    tokenizer
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [26]:
model_path = "/content/drive/MyDrive/data/text-transfer-smilegate-bart-eos/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=24, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps=500, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [27]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,3.485100,2.596757
1000,2.597400,2.368720
1500,2.406500,2.264215
2000,2.253300,2.194721
2500,2.140500,2.154789
3000,2.087500,2.099530
3500,2.061400,2.080022
4000,1.904400,2.047471
4500,1.883800,2.018365
5000,1.877000,1.999570


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

KeyboardInterrupt: ignored

In [28]:
trainer.save_model("/content/drive/MyDrive/bible/model/")

In [29]:
from transformers import pipeline

nlg_pipeline = pipeline('text2text-generation',model="/content/drive/MyDrive/bible/model/", tokenizer=model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [30]:
def generate_text(pipe, text, target_style, num_return_sequences=5, max_length=60):
  target_style_name = style_map[target_style]
  text = f"{target_style_name} 말투로 변환:{text}"
  out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length)
  return [x['generated_text'] for x in out]


In [31]:
target_styles = df.columns
src_text = """
어쩌다 마주친 그대 모습에
내 마음을 빼앗겨 버렸네
어쩌다 마주친 그대 두 눈이
내 마음을 사로잡아 버렸네
그대에게 할 말이 있는데
왜 이리 용기가 없을까
음 말을 하고 싶지만 자신이 없어
내 가슴만 두근두근
답답한 이 내 마음
바람 속에 날려 보내리
"""

print("입력 문장:", src_text)
for style in target_styles:
  print(style, generate_text(nlg_pipeline, src_text, style, num_return_sequences=1, max_length=1000)[0])

입력 문장: 
어쩌다 마주친 그대 모습에
내 마음을 빼앗겨 버렸네
어쩌다 마주친 그대 두 눈이
내 마음을 사로잡아 버렸네
그대에게 할 말이 있는데
왜 이리 용기가 없을까
음 말을 하고 싶지만 자신이 없어
내 가슴만 두근두근
답답한 이 내 마음
바람 속에 날려 보내리

input_text 내가 어찌 보고 네 얼굴을 바라보았느냐 내 눈이 네 눈을 바라보았느냐 내가 네게 무슨 말을 할꼬 내가 어찌하여 네게 대답하기를 주저하였느냐 내가 어찌하여 네게 대답하기를 주저하였느냐
target_text 내가 언제친대 얼굴을 보고 마음이 빼앗겼구나. 내가 친히 보는 눈이 나를 사로잡았구나. 내가 무슨 말을 할 수 있겠는가? 내가 무슨 말을 할 수 있겠는가? 내가 무슨 말을 할 수 있겠는가?


In [48]:
df.columns

Index(['input_text', 'target_text'], dtype='object')

In [68]:
target_styles = df.columns
src_text = """
날 설득해봐. 네가 부활을 해야 할 이유를 납득시켜보라고.
"""

print("입력 문장:", src_text)
print( "성경체: ", generate_text(nlg_pipeline, src_text,"input_text", num_return_sequences=1, max_length=1000)[0])

#target_text 는 문어체
# input_text 는 성경체

입력 문장: 
날 설득해봐. 네가 부활을 해야 할 이유를 납득시켜보라고.

성경체:  나를 설득하라 네가 부활할 것을 내게 알게 하라


In [66]:
target_styles = df.columns
src_text = """
내가 한강을 갔더니 하늘에서 비가 내렸다.
"""

print("입력 문장:", src_text)
print( "성경체: ", generate_text(nlg_pipeline, src_text,"input_text", num_return_sequences=1, max_length=1000)[0])


입력 문장: 
내가 한강을 갔더니 하늘에서 비가 내렸다.

성경체:  내가 강으로 가니 하늘에서 비 내리는 것이 보였고


In [69]:

target_styles = df.columns
src_text = """
안녕하세요 저는 오늘 무척 우울한 상태입니다
"""

print("입력 문장:", src_text)
print( "성경체: ", generate_text(nlg_pipeline, src_text,"input_text", num_return_sequences=1, max_length=1000)[0])


입력 문장: 
안녕하세요 저는 오늘 무척 우울한 상태입니다

성경체:  안녕하 저는 오늘 심히 근심하나이다


In [70]:
target_styles = df.columns
src_text = """
안녕하세요 저는 오늘 무척 우울한 상태입니다.
"""

print("입력 문장:", src_text)
print( "성경체: ", generate_text(nlg_pipeline, src_text,"input_text", num_return_sequences=1, max_length=1000)[0])


입력 문장: 
안녕하세요 저는 오늘 무척 우울한 상태입니다.

성경체:  안녕하 저는 오늘 심히 근심하나이다


In [71]:
target_styles = df.columns
src_text = """
안녕하세요 저는 오늘 무척 우울한 상태입니다!
"""

print("입력 문장:", src_text)
print( "성경체: ", generate_text(nlg_pipeline, src_text,"input_text", num_return_sequences=1, max_length=1000)[0])


입력 문장: 
안녕하세요 저는 오늘 무척 우울한 상태입니다!

성경체:  안녕하 저는 오늘 심히 근심하나이다
